In [1]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd

In [2]:
if torch.cuda.is_available():  
  dev = "cuda:0" 
else:  
  dev = "cpu"  
device = torch.device(dev)
torch.use_deterministic_algorithms(True)
np.random.seed(15) # if numpy is used
torch.manual_seed(15)
if torch.cuda.is_available():
    torch.cuda.manual_seed(15)

In [3]:
df = pd.read_excel("data.xlsx", sheet_name="data")
df.head()

,From Date,PM2.5,PM10,NO,NO2,NOx,NH3,SO2,CO,Ozone,Benzene
0,2019-01-01 00:00:00,0.548004,0.837675,0.386063,0.183578,0.377457,0.311067,0.099945,0.59,0.008504,0.391534
1,2019-01-01 00:15:00,0.548004,0.837675,0.447938,0.177646,0.426394,0.304088,0.088901,0.60,0.011006,0.373016
2,2019-01-01 00:30:00,0.548004,0.837675,0.385062,0.201686,0.382471,0.333500,0.088349,0.52,0.011006,0.365079
3,2019-01-01 00:45:00,0.448759,0.658317,0.440328,0.196691,0.426394,0.331007,0.085036,0.54,0.011006,0.359788
4,2019-01-01 01:00:00,0.448759,0.658317,0.422307,0.175148,0.404332,0.327517,0.083932,0.50,0.011131,0.333333


In [4]:
df = df.set_index('From Date')

In [5]:
test_data_size = int(df.shape[0]/4)
train_data = df[:-test_data_size]
test_data = df[-test_data_size:]
train_data.shape, test_data.shape

((52632, 10), (17544, 10))

In [6]:
def create_inout_sequences(input_data, tw):
    inout_seq = []
    L = len(input_data)
    for i in range(L-tw):
        train_seq = input_data[i:i+tw]
        train_label = input_data[i+tw]
        inout_seq.append((train_seq ,train_label))
    return inout_seq

In [7]:
test_data_tensor = torch.FloatTensor(test_data.values).view(-1,10).to(device)

test_inout_seq = create_inout_sequences(test_data_tensor, 16)
test_inout_seq[0]


(tensor([[0.0658, 0.1263, 0.0094, 0.0952, 0.0395, 0.0962, 0.0304, 0.1000, 0.1371,
          0.0741],
         [0.0658, 0.1263, 0.0096, 0.0924, 0.0387, 0.0967, 0.0331, 0.1000, 0.1381,
          0.0741],
         [0.0658, 0.1263, 0.0098, 0.0815, 0.0351, 0.0962, 0.0387, 0.1000, 0.1381,
          0.0794],
         [0.0593, 0.1513, 0.0110, 0.0871, 0.0381, 0.0982, 0.0447, 0.1000, 0.1311,
          0.0899],
         [0.0593, 0.1513, 0.0172, 0.0871, 0.0431, 0.1017, 0.0414, 0.1100, 0.1286,
          0.0926],
         [0.0593, 0.1513, 0.0128, 0.0884, 0.0399, 0.1122, 0.0409, 0.1100, 0.1576,
          0.0952],
         [0.0593, 0.1513, 0.0124, 0.0902, 0.0401, 0.1181, 0.0375, 0.1100, 0.1746,
          0.1111],
         [0.0626, 0.1503, 0.0118, 0.0974, 0.0423, 0.1181, 0.0375, 0.1100, 0.1896,
          0.1164],
         [0.0626, 0.1503, 0.0174, 0.1140, 0.0525, 0.1171, 0.0370, 0.1200, 0.1801,
          0.1164],
         [0.0626, 0.1503, 0.0286, 0.1146, 0.0618, 0.1142, 0.0359, 0.1200, 0.2021,
         

In [8]:
test_inout_seq[0][0]

tensor([[0.0658, 0.1263, 0.0094, 0.0952, 0.0395, 0.0962, 0.0304, 0.1000, 0.1371,
         0.0741],
        [0.0658, 0.1263, 0.0096, 0.0924, 0.0387, 0.0967, 0.0331, 0.1000, 0.1381,
         0.0741],
        [0.0658, 0.1263, 0.0098, 0.0815, 0.0351, 0.0962, 0.0387, 0.1000, 0.1381,
         0.0794],
        [0.0593, 0.1513, 0.0110, 0.0871, 0.0381, 0.0982, 0.0447, 0.1000, 0.1311,
         0.0899],
        [0.0593, 0.1513, 0.0172, 0.0871, 0.0431, 0.1017, 0.0414, 0.1100, 0.1286,
         0.0926],
        [0.0593, 0.1513, 0.0128, 0.0884, 0.0399, 0.1122, 0.0409, 0.1100, 0.1576,
         0.0952],
        [0.0593, 0.1513, 0.0124, 0.0902, 0.0401, 0.1181, 0.0375, 0.1100, 0.1746,
         0.1111],
        [0.0626, 0.1503, 0.0118, 0.0974, 0.0423, 0.1181, 0.0375, 0.1100, 0.1896,
         0.1164],
        [0.0626, 0.1503, 0.0174, 0.1140, 0.0525, 0.1171, 0.0370, 0.1200, 0.1801,
         0.1164],
        [0.0626, 0.1503, 0.0286, 0.1146, 0.0618, 0.1142, 0.0359, 0.1200, 0.2021,
         0.1111],
        [0

In [9]:
class LSTM(nn.Module):
    def __init__(self, input_size=10, hidden_layer_size=100, output_size=10):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        self.lstm = nn.LSTM(input_size, hidden_layer_size)

        self.linear = nn.Linear(hidden_layer_size, output_size)

        self.hidden_cell = (torch.zeros(1,1,self.hidden_layer_size).to(device),
                            torch.zeros(1,1,self.hidden_layer_size).to(device))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq) ,1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]

In [10]:
model = torch.load("model.pt").to(device)
model.eval()


LSTM(
  (lstm): LSTM(10, 100)
  (linear): Linear(in_features=100, out_features=10, bias=True)
)

In [11]:
y_pred = model(test_inout_seq[0][0])
y_pred,test_inout_seq[0][1]

(tensor([0.0584, 0.2129, 0.0031, 0.0484, 0.0206, 0.0869, 0.0328, 0.0843, 0.4717,
         0.0662], device='cuda:0', grad_fn=<SelectBackward>),
 tensor([0.0507, 0.2114, 0.0062, 0.0490, 0.0211, 0.0922, 0.0364, 0.1000, 0.4647,
         0.0688], device='cuda:0'))

In [50]:
model.eval()

LSTM(
  (lstm): LSTM(10, 100)
  (linear): Linear(in_features=100, out_features=10, bias=True)
)

In [19]:
test_inout_seq[0][1]

tensor([0.0507, 0.2114, 0.0062, 0.0490, 0.0211, 0.0922, 0.0364, 0.1000, 0.4647,
        0.0688], device='cuda:0')

In [45]:
with torch.no_grad():
    model.eval()  
    model.hidden = (torch.zeros(1, 1, model.hidden_layer_size).to(device),torch.zeros(1, 1, model.hidden_layer_size).to(device))
    y_pred = model(test_inout_seq[0][0])
y_pred,test_inout_seq[0][1]

(tensor([ 0.0552,  0.1489, -0.1106,  0.0369, -0.0749,  0.1468,  0.0415,  0.0051,
          0.5685,  0.0716], device='cuda:0'),
 tensor([0.0507, 0.2114, 0.0062, 0.0490, 0.0211, 0.0922, 0.0364, 0.1000, 0.4647,
         0.0688], device='cuda:0'))

In [12]:
y_pred.shape, test_inout_seq[0][1].shape

(torch.Size([10]), torch.Size([10]))

In [13]:


y_test = test_inout_seq[0][1]

r2_score(y_test.cpu().detach(), y_pred.cpu().detach())

0.9975537798614699

In [40]:
y_pred_list = []

with torch.no_grad():
    model.eval()
    for seq, _ in test_inout_seq:
        model.hidden = (torch.zeros(1, 1, model.hidden_layer_size),torch.zeros(1, 1, model.hidden_layer_size))
        y_test_pred = model(seq)
        y_pred_list.append(y_test_pred.cpu().detach().numpy())


In [61]:
seq1 =  [tupl[0] for tupl in test_inout_seq]
seq1[0]

tensor([[0.0658, 0.1263, 0.0094, 0.0952, 0.0395, 0.0962, 0.0304, 0.1000, 0.1371,
         0.0741],
        [0.0658, 0.1263, 0.0096, 0.0924, 0.0387, 0.0967, 0.0331, 0.1000, 0.1381,
         0.0741],
        [0.0658, 0.1263, 0.0098, 0.0815, 0.0351, 0.0962, 0.0387, 0.1000, 0.1381,
         0.0794],
        [0.0593, 0.1513, 0.0110, 0.0871, 0.0381, 0.0982, 0.0447, 0.1000, 0.1311,
         0.0899],
        [0.0593, 0.1513, 0.0172, 0.0871, 0.0431, 0.1017, 0.0414, 0.1100, 0.1286,
         0.0926],
        [0.0593, 0.1513, 0.0128, 0.0884, 0.0399, 0.1122, 0.0409, 0.1100, 0.1576,
         0.0952],
        [0.0593, 0.1513, 0.0124, 0.0902, 0.0401, 0.1181, 0.0375, 0.1100, 0.1746,
         0.1111],
        [0.0626, 0.1503, 0.0118, 0.0974, 0.0423, 0.1181, 0.0375, 0.1100, 0.1896,
         0.1164],
        [0.0626, 0.1503, 0.0174, 0.1140, 0.0525, 0.1171, 0.0370, 0.1200, 0.1801,
         0.1164],
        [0.0626, 0.1503, 0.0286, 0.1146, 0.0618, 0.1142, 0.0359, 0.1200, 0.2021,
         0.1111],
        [0

In [33]:
col1 = [tupl[1] for tupl in test_inout_seq]
col1 = [a.cpu().numpy() for a in col1]
col1[0]

array([0.05070119, 0.21142285, 0.00620745, 0.04901655, 0.02105897,
       0.09222333, 0.03644395, 0.1       , 0.46473238, 0.06878307],
      dtype=float32)

In [60]:
predlis = model(seq1)

AttributeError: 'list' object has no attribute 'view'

In [35]:
model(test_inout_seq[0][0])

tensor([-0.0024, -0.0068, -0.1335, -0.0616, -0.1294,  0.1465,  0.0050, -0.0153,
         0.5077, -0.0203], device='cuda:0', grad_fn=<SelectBackward>)

In [42]:
r_square = r2_score(col1, y_pred_list)

print("R^2 :",r_square)

R^2 : 0.008063888902471051


In [27]:
col1[0], y_pred_list[0]



([0.05070118606090546,
  0.2114228457212448,
  0.006207448896020651,
  0.049016546458005905,
  0.02105896547436714,
  0.09222333133220673,
  0.03644395247101784,
  0.10000000149011612,
  0.4647323787212372,
  0.06878306716680527],
 [0.05478591471910477,
  0.20234224200248718,
  -0.0028618574142456055,
  0.047275230288505554,
  0.015922367572784424,
  0.08844872564077377,
  0.030655428767204285,
  0.0806216448545456,
  0.4856809973716736,
  0.065723717212677])